# Tutorial 6: Motion planning

In this tutorial, we are going to plan some trajectories of the robot in the 2-D task space. And then, we are going to will use the tools that we have built in the previous tutorials to manipulate the robot accordingly.

Motion planning takes place in both task space and the C-space. Usually, it tries to find a smooth path from the starting position to some target positions. 

Outline of this tutorial

1. Polynomial interpolation 
2. Planning the polynomial trajectory 
3. Execute the trajectory (no coding required)

Loading some code from the previous tutorials.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML

class ThreeLinkArm:
    
    def __init__(self):
        # Set parameters for the 3-link planar arm
        self.theta1 = np.deg2rad(0)
        self.theta2 = np.deg2rad(90)
        self.theta3 = np.deg2rad(90)
        self.l1 = 5
        self.l2 = 3
        self.l3 = 2

    def forwardKinematics(self, theta1, theta2, theta3):
        # Define the homogeneous transformation matrices for the 3-link planar arm
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.t01 = np.matrix([[np.cos(self.theta1), -np.sin(self.theta1), 0], [np.sin(self.theta1), np.cos(self.theta1), 0], [0, 0, 1]])
        self.t12 = np.matrix([[np.cos(self.theta2), -np.sin(self.theta2), self.l1], [np.sin(self.theta2), np.cos(self.theta2), 0], [0, 0, 1]])
        self.t23 = np.matrix([[np.cos(self.theta3), -np.sin(self.theta3), self.l2], [np.sin(self.theta3), np.cos(self.theta3), 0], [0, 0, 1]])        
        self.t3end = np.matrix([[np.cos(self.theta3), -np.sin(self.theta3), self.l3], [np.sin(self.theta3), np.cos(self.theta3), 0], [0, 0, 1]])
        self.t0end = self.t01*self.t12*self.t23*self.t3end
        return self.t0end
    
    def findJointPos(self): 
        # Find the x,y position of each joint and end effector so it can be plotted
        # Find the transformation matrices for joint 2 and joint 3
        self.t02 = self.t01*self.t12
        self.t03 = self.t01*self.t12*self.t23
        # Find the x, y coordinates for joints 2 and 3. Put them in a list j2 = [x,y]
        j2 = [ self.t02[0, 2], self.t02[1, 2] ]
        j3 = [ self.t03[0, 2], self.t03[1, 2] ]
        endeff = [self.t0end[0,2],self.t0end[1,2]]
        
        self.jnt2pos, self.jnt3pos, self.endEffPos = j2,j3,endeff
        return j2,j3,endeff
    
    def geomJacobian(self, jnt2pos=None, jnt3pos=None, endEffPos=None):
        if jnt2pos is None:
            jnt2pos=self.jnt2pos 
        if jnt3pos is None:
            jnt3pos=self.jnt3pos
        if endEffPos is None:
            endEffPos=self.endEffPos
        ai = np.array([0,0,1])
        col0 = np.array(endEffPos + [0])
        col1 = np.array(endEffPos + [0]) - np.array(jnt2pos + [0])
        col2 = np.array(endEffPos + [0]) - np.array(jnt3pos + [0])
        J = np.array([np.cross(ai,col0), np.cross(ai,col1), np.cross(ai,col2)]).T 
        return J

def plotArm(jnt2pos, jnt3pos, endEffectPos, target=None, step=None):
    # set up figure
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-10, 10), ylim=(-10, 10))
    ax.grid()
    
    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')
        
    line, = ax.plot([], [], 'o-', lw=4, mew=5)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], endEffectPos[1]])
    line.set_data((x,y))
    
    plt.show()
    
# gif animation plotter
def plotArm2(jnt2pos, jnt3pos, endEffectPos, target=None, step=None, fig=None, camera=None):
    # set up figure
    ax = fig.add_subplot(111, autoscale_on=False,
                         xlim=(-10, 10), ylim=(-10, 10))
    ax.grid()
    line, = ax.plot([], [], 'o-', lw=4, mew=5, color='lightblue')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    line.set_data([], [])
    time_text.set_text('')
    x = np.array([0, jnt2pos[0], jnt3pos[0], endEffectPos[0]])
    y = np.array([0, jnt2pos[1], jnt3pos[1], endEffectPos[1]])
    line.set_data((x,y))
    
    if step is not None:
        step = np.array(step)
        plt.scatter(step[:, 0],step[:, 1], color='lightblue', marker='o')
        plt.plot(step[:, 0],step[:, 1], color='lightblue')
    if target is not None:
        target = np.array(target)
        plt.scatter(target[:, 0],target[:, 1], color='red', marker='o')

    camera.snap()

# Part 1:  Polynomial interpolation 

Given there are $n+1$ points $(x_1, y_1), (x_2, y_2), \ldots, (x_n, y_n), (x_n+1, y_n+1)$ with distinct $x_i$ values, we want to fit a polynomial, $f$, that passes through those points, i.e. for each point $x_i$, there is $f(x_i) = y_i$. For example, there are 5 points:

<img src="images/points.png" style="width:300px"/>

By the theorem of interpolation, there exists a unique polynomial of degree at most $n$ under the assumptions above. Se, lets fit a polynomial of degree 4.

<img src="images/points_interp.png" style="width:300px"/>

The answer is $f(x) = -75 + 81 x  + -29.25 x^2 + 4.5 x^3 - 0.25 x^4$

In [ ]:
np.polyfit(np.array([2,3,4,5,6]),np.array([2,6,5,5,6]),4) 
# Note: the coefficients are in the order of the highest order to the lowest order. 

In [ ]:
f = lambda x: np.hstack([np.ones((x.shape[0], 1)), x, x**2, x**3, x**4])
plot_x = np.arange(1.8, 6.2, 0.01)[:, None]
plot_y = f(plot_x) @ np.flipud(np.array([-0.25, 4.5, -29.25, 81., -75.]))

plt.figure(figsize=(6,4))
plt.plot(plot_x, plot_y)
plt.scatter(np.array([2,3,4,5,6]),np.array([2,6,5,5,6])) 
plt.show()

Therefore, in the upcoming exercises, let's use $N-1$ order polynomial for interpolating the $N$ points. Assume the input values $x$ are unique.

# Part 2: Trajectory planning

In this part, let's plan some simple polynomial trajectories in the work space. 

First, setup a simple environment with the arm manipulator and four intermediate interpolation points.

In [ ]:
np.set_printoptions(precision = 2) 
np.set_printoptions(suppress = True) # remove exponential notation

# Setup our new manipulator
arm = ThreeLinkArm()

# define the initial configuration of the manipulator (you are welcomed to try other values)
initTheta = [np.deg2rad(0),np.deg2rad(45),np.deg2rad(-45)]

# Initialise the arm with the initial values
_ = arm.forwardKinematics(*initTheta)

# Get the initial end-effector position 
init_joint2pos, init_joint3pos, EF_init_pos = arm.findJointPos()

# Define the target postions of the end-effector
EF_target_pos = np.array([[EF_init_pos[0], 6.,2.,-4.,-8.],
                          [EF_init_pos[1], -6.,-7.,-5.,-1.]]).T

# Plot the pose of the arm
plotArm(*arm.findJointPos(), target=EF_target_pos)
print(f"The interpolation points are stored in the variable \"EF_target_pos\": \n{EF_target_pos}")

#### Now, let's interpolate the trajectory of the end-effector using the polynomials, you can use numpy.polyfit() function here or any other method that you would prefer.

In [ ]:
# Interpolate the polynomial 
### START YOUR CODE HERE
pol = None # fix me!
### END OF YOUR CODE
assert pol is not None, "Please interpolate the polynomial above."

In [ ]:
# Plotting the planned trajectory
xx = np.linspace(EF_target_pos[0, 0], EF_target_pos[-1, 0], 20)
yy = np.polyval(pol,xx)   

epsilon = 1e-2
xx = np.arange(EF_target_pos[0,0],EF_target_pos[-1,0], -epsilon)
yy = np.polyval(pol,xx)   
curve = np.hstack([xx[:, None], yy[:, None]]) 
total_arc_length = np.sum(np.linalg.norm(curve[:-1, None] - curve[1:, None], axis=1))

interpolation_threshold = 0.8
interpolated_points = np.array([curve[0, :]])

# Extract the interpolation points with the same arc lengths
last_point = curve[0, :]
for p in curve[1:, :]:
    if np.linalg.norm(p - interpolated_points[-1, :]) > interpolation_threshold:
        interpolated_points = np.vstack([interpolated_points, last_point])
    last_point = p
interpolated_points = np.vstack([interpolated_points, curve[-1, :]])    

plotArm(*arm.findJointPos(), target=EF_target_pos, step=interpolated_points)
print(f"The intermediate points are stored in the variable \"interpolated_points\"")

# Part 3: Executing the planned motion

No coding required here. But make sure you are using the correct variables. By running the code below, you should see a GIF image of the planned motion.

In [ ]:
# set the current E-F position for the IK loop
joint2pos, joint3pos, endEffectorPos = init_joint2pos, init_joint3pos, EF_init_pos
# set the initial robot configuration
newTheta = initTheta

# Set up the animation generator
fig = plt.figure(figsize=(4,4))
camera = Camera(fig)

for i in range(interpolated_points.shape[0]):
    J = arm.geomJacobian()
    
    newgoal = interpolated_points[i, :]
    deltaStep = newgoal - endEffectorPos
    subtarget = np.array([deltaStep[0], deltaStep[1], 0]) 

    radTheta = np.linalg.pinv(J) @ subtarget
    newTheta = newTheta + radTheta

    # ----------- Do forward kinematics to plot the arm ---------------
    _ = arm.forwardKinematics(newTheta[0],newTheta[1],newTheta[2])
    joint2pos, joint3pos, endEffectorPos = arm.findJointPos()
    plotArm2(joint2pos, joint3pos, endEffectorPos, target=EF_target_pos, fig=fig, camera=camera)

plt.close()
animation = camera.animate()
animation.save('animated_motion.gif', writer = 'imagemagick')

# show the animation
from IPython.display import HTML
HTML('<img src="animated_motion.gif">')
# Ignore the warning message